# 2. Deep_dive_missing_data

When first plotting the data in Tableau, one can observe: 
1) for year 2016-2018, the number of cases are as doubled compared to the preious years and 2019. 
2) for year 2016-2018, almost half of the records didn't contain complete information, especially for SERVICE CODE, location, species, breed etc.

The hypothesis is:
--->REQUEST CODE and COMPLAINANT_STREET contains information collected during the emergency phone call. 

--> However, only if the Baton Rouge Animal Control and Rescue Center (ACRC) provided service ("SERVICE CODE"), more detailed information such as breed, temperment, condition, species and location are collected. 

--> Empty SERVICE CODE fields only appeared in the years 2016-2018 and the systems has been changed back to only recordings cases, where service has been provided. 

--> Therefore, as a consequence, I decided to take out all records in the years 2016-2019, whenever the "SERVICE CODE" is not filled out. 

To test my hypothesis, I used the table with all relevant columns and did several cross-checks. 

In [1]:
import pandas as pd

In [2]:
df = pd.read_pickle("/Users/Tianwang/ironhack/git/03_project_ironhack_tableau/data/baton_rouge_animal_control_incidents_deep_dive.pkl")

In [3]:
#SERVICE CODE = NaN / Hypothesis: REQUEST TYPE IS NOT empty. --> proven

df[df["SERVICE CODE"].isnull()]["REQUEST TYPE"].value_counts(dropna = False)

UNKNOWN                       4719
STRAY                         2077
LOOSE ANIMAL                  1552
CRUELTY CASE                  1439
DELIVERED TAGS                1065
OTHER                          953
PICK UP TRAP                   784
BITE CASE                      654
PATROL AREA                    654
PIT                            618
CRUELTY RECHECK                509
ANIMAL RESCUE                  499
DELIVER TRAP                   479
ABANDONED DOG                  429
TRAPPED ANIMAL                 390
STRAY ANIMAL SECURED           381
SPECIAL CASE                   373
INJURED ANIMAL                 360
PICK UP OR DELIVER PACKAGE     349
SNAKE                          338
DANGEROUS ANIMAL SITUATION     329
WILD LIFE                      290
UNIT STANDING BY               208
BARKING DOG                    201
LOOSE LIVE STOCK               168
ANIMAL ON SCHOOL CAMPUS         91
SICK ANIMAL                     83
OWNER SIGNED RELEASE            81
VET PICKUP          

In [4]:
df["SERVICE CODE"].value_counts(dropna = False) 

NaN                     20193
TRAPPED ANIMAL          17585
STRAY                    8227
STRAY AND SECURED        8067
LOOSE                    2742
OWNER SIGNED RELEASE     2193
INJURED                  2001
ABANDONED ANIMAL         1904
WILD LIVE                1313
CRUELTY                  1186
UNIT STANDING BY         1156
BITE CASE LOOSE           932
DANGEROUS ANIMAL SIT      844
ANIMAL RESCUE             840
SICK ANIMAL               801
VET PICKUP                720
BITE CASE SECURE          452
SNAKE                     342
LOOSE LIVESTOCK           330
SPECIAL CASE              312
UNKNOWN                   260
SCHOOL CALL               258
DOG FIGHTING              131
BITE CASE                 130
OWNED                       4
HIT BY CAR                  1
Name: SERVICE CODE, dtype: int64

In [5]:
df["SPECIES"].value_counts(dropna = False) #hypothesis: if Species = NaN, Service Code is empty as well.

DOG          26700
NaN          20215
CAT          13239
WILDLIFE     11157
LIVESTOCK      465
REPTILE        415
FOWL           339
BIRD           175
BIRDS          129
UNKNOWN         43
OTHER           41
EXOTIC           6
Name: SPECIES, dtype: int64

In [6]:
df[df["SPECIES"].isnull()]["SERVICE CODE"].value_counts(dropna = False) #hypothesis proven. 

NaN                     20193
TRAPPED ANIMAL              9
WILD LIVE                   3
DANGEROUS ANIMAL SIT        2
ABANDONED ANIMAL            2
VET PICKUP                  1
STRAY                       1
INJURED                     1
OWNER SIGNED RELEASE        1
UNIT STANDING BY            1
CRUELTY                     1
Name: SERVICE CODE, dtype: int64

In [7]:
df["SERVICE CODE"].value_counts(dropna = False) 
#hypothesis: if SERVICE CODE is NaN, all other relevant fields are NaN as well.(e.g. Breed, Species, Temperment, Condition)

NaN                     20193
TRAPPED ANIMAL          17585
STRAY                    8227
STRAY AND SECURED        8067
LOOSE                    2742
OWNER SIGNED RELEASE     2193
INJURED                  2001
ABANDONED ANIMAL         1904
WILD LIVE                1313
CRUELTY                  1186
UNIT STANDING BY         1156
BITE CASE LOOSE           932
DANGEROUS ANIMAL SIT      844
ANIMAL RESCUE             840
SICK ANIMAL               801
VET PICKUP                720
BITE CASE SECURE          452
SNAKE                     342
LOOSE LIVESTOCK           330
SPECIAL CASE              312
UNKNOWN                   260
SCHOOL CALL               258
DOG FIGHTING              131
BITE CASE                 130
OWNED                       4
HIT BY CAR                  1
Name: SERVICE CODE, dtype: int64

In [8]:
test = df.loc[df["SERVICE CODE"].isnull(),["SPECIES","TEMPERMENT","CONDITION"]]

In [9]:
test["SPECIES"].value_counts(dropna=False)

NaN    20193
Name: SPECIES, dtype: int64

In [10]:
test["TEMPERMENT"].value_counts(dropna=False)

NaN    20193
Name: TEMPERMENT, dtype: int64

In [11]:
test["CONDITION"].value_counts(dropna=False)

NaN    20193
Name: CONDITION, dtype: int64

# Remove records, where no service has been provided.

In [12]:
df_select = pd.read_pickle("/Users/Tianwang/ironhack/git/03_project_ironhack_tableau/data/baton_rouge_animal_control_incidents_clean.pkl")

In [27]:
df_select.head()

,file_number,incident_date,complainant_street,location,municipality,zip_code,service_code,species,breed,condition,...,arrival_time_new,available_time_new,incident_time_new,dispatched_time_new,all_time_available,incident_to_dispatch,disp_to_arrival,service_code_summary,species_clean,clean_adress
0,201639027,12/27/2016,SYCAMORE ST,3366 SYCAMORE,BATON ROUGE,70805,LOOSE,DOG,PITBULL,FAIR,...,2016-12-27 16:17:00,2016-12-27 16:38:00,2016-12-27 15:30:00,2016-12-27 16:06:00,Yes,36.0,11.0,LOOSE,DOG,"3366 SYCAMORE, BATON ROUGE, 70805"
1,201639044,12/28/2016,WOODLAND RIDGE BL,2582 WOODLAND RIDGE,BATON ROUGE,70816,TRAPPED ANIMAL,WILDLIFE,POSSUM,FAIR,...,2016-12-28 10:15:00,2016-12-28 10:23:00,2016-12-28 10:06:00,2016-12-28 10:07:00,Yes,1.0,8.0,TRAPPED ANIMAL,WILDLIFE,"2582 WOODLAND RIDGE, BATON ROUGE, 70816"
2,201639036,12/28/2016,MAIN ST,4463 MAIN,ZACHARY,70791,STRAY AND SECURED,DOG,PITBULL,EMACIATED,...,2016-12-28 09:27:00,2016-12-28 09:45:00,2016-12-28 09:03:00,2016-12-28 09:03:00,Yes,0.0,24.0,STRAY,DOG,"4463 MAIN, ZACHARY, 70791"
3,201639042,12/28/2016,ERIE ST,2100 ERIE,BATON ROUGE,70805,ABANDONED ANIMAL,DOG,GERMAN SHEPHERD,EMACIATED,...,2016-12-28 10:35:00,2016-12-28 10:47:00,2016-12-28 09:46:00,2016-12-28 10:24:00,Yes,38.0,11.0,ABANDONED ANIMAL,DOG,"2100 ERIE, BATON ROUGE, 70805"
4,201639059,12/28/2016,VAN BUREN ST,299 VAN BUREN,BATON ROUGE,70714,STRAY,DOG,YORKSHIRE TERRIER,EMACIATED,...,2016-12-28 16:07:00,2016-12-28 16:25:00,2016-12-28 15:05:00,2016-12-28 15:45:00,Yes,40.0,22.0,STRAY,DOG,"299 VAN BUREN, BATON ROUGE, 70714"


## First returning a new dataframe with only rows, where service_code is not NaN

In [35]:
new = df_select.loc[df_select["service_code"].notna()]

In [36]:
len(new) 

52731

In [37]:
len(df_select) #the old df hat 72924 rows.

72924

In [38]:
to_be_dropped = len(df_select) - len(test)
to_be_dropped 

#20193 rows with emtpy Service Code will be dropped.

52731

## Cross-check how the dropped rows look like. 

In [39]:
isna = df[df_select["service_code"].isna()]

In [40]:
isna["year"].value_counts() #Just to confirm that the insa values almost only occured in 2016-2018. 

2017    9154
2016    6544
2018    4486
2019       9
Name: year, dtype: int64

## Create a new file with the new dataframe.

In [41]:
pd.to_pickle (new, "/Users/Tianwang/ironhack/git/03_project_ironhack_tableau/data/baton_rouge_animal_control_incidents_clean_update.pkl")

In [42]:
new.to_csv("/Users/Tianwang/ironhack/git/03_project_ironhack_tableau/data/baton_rouge_animal_control_incidents_clean_update.csv", sep=";", index=False)